In [3]:
import pandas as pd
import numpy as np
import  requests
from bs4 import BeautifulSoup
import csv
import time

In [49]:
l=[]

In [50]:
def scrapper123(url1):
    try:
        HEADERS = ({
            'authority': 'https://www.amazon.co.uk',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0',
            'Sec-Ch-Ua-Platform': 'Windows'
        })
       
        response1=requests.get(url1,headers=HEADERS)
        response1.raise_for_status()
        soup1=BeautifulSoup(response1.text,"html.parser")
        #print(soup1)
        listing1=soup1.find_all('div',{'class':'sg-col-4-of-24'})
        #print(listing1)
        for list in listing1:
            try:
                #information={}
                
                product_url=list.find('div',{'class':'s-product-image-container'}).find('a',{'class':'s-no-outline'})['href']
                product_link="https://www.amazon.co.uk"+product_url
                #information['product_url']=product_link
                print(product_link)
                
                product_image=list.find('div',{'class':'s-product-image-container'}).find('a',{'class':'s-no-outline'}).find('img',{'class':'s-image'})['src']
                

                
                response_product=requests.get(product_link,headers=HEADERS)
                soup_product=BeautifulSoup(response_product.text,"html.parser")
                
                
                #title of product
                product_title=soup_product.find('div',{'class':'a-container'}).find('div',{'id':'title_feature_div'}).find('h1',{'id':'title'}).find('span',{'id':'productTitle'}).get_text().strip()
                print("Product title is : "+product_title)
                #information['product_title']=product_title

                #product ASIN
                #product_asin=soup_product.find('div',{'id':'productDetails_db_sections'}).find('td',{'class':'prodDetAttrValue'}).get_text().strip()
                product_asin_url=product_link.split('/')
                asin=product_asin_url[5]
                #information['asin']=asin
                #print("ASIN is : "+asin)
                
                #store name of product
                product_store=soup_product.find('div',{'id':'bylineInfo_feature_div'}).find('a',{'class':'a-link-normal'}).get_text().strip().replace("Visit the ","")
                #print("Product store name is : "+product_store)
                #information['product_store']=product_store


                #product rating out of 5
                #product_rating=soup_product.find('div',{'id':'averageCustomerReviews_feature_div'}).find('span',{'class':'a-color-base'}).get_text().strip()
                product_rating=soup_product.find('i',{'class':'cm-cr-review-stars-spacing-big'}).get_text().strip().replace(" out of 5 stars","")
                #print("Product rating is : "+product_rating)
                #information['product_rating']=product_rating

                #Product Selling Price
                #product_price_whole=soup_product.find('span',{'class':'a-price'}).find('span',{'class':'a-price-whole'}).get_text().strip().replace(".","")
                #product_price_fraction=soup_product.find('span',{'class':'a-price'}).find('span',{'class':'a-price-fraction'}).get_text().strip()
                #product_price_whole=soup_product.find('span',{'class':'a-price'}).get_text().strip()
                #print("PPPRRRIIICCEEE : "+product_price_whole)
                #print("Selling price is : "+product_price_whole+"."+product_price_fraction)
                
                #link for all reviews page 
                all_reviews=soup_product.find('div',{'id':'reviews-medley-footer'}).find('a',{'class':'a-link-emphasis'})['href']
                all_reviews_link="https://www.amazon.co.uk"+all_reviews
                #information['all_reviews_link']=all_reviews_link
                #print("See all reviews link: "+all_reviews_link) 

                #l.append(information)
                
                #function call
                reviews_scrapper(all_reviews_link,product_title,product_image,asin,product_rating,product_store,product_link)
                
                #information['reviews']=review_array
                
                #print(l)

            
            except:
                #print("***************error found while printing product details*************")
                pass
        try:
            print("---------------------------------------------------------------------------")
            next_page1=soup1.find('a',{'class':'s-pagination-next'})['href']
            link1="https://www.amazon.co.uk"+next_page1
            print(link1)
            #scrapper123(link1)
        except:
            print("***NO MORE PAGES***")

    
    
    except requests.exceptions.HTTPError as err:
        if response1.status_code==503:
            print("HTTP 503 error occured")
            #time.sleep(60)
            #scrapper123(url1)




def reviews_scrapper(all_reviews_link,product_title,product_image,asin,product_rating,product_store,product_link):
    HEADERS = ({
            'authority': 'https://www.amazon.co.uk',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0',
            'Sec-Ch-Ua-Platform': 'Windows'
        })

    
    #****all reviews page information*****
    reviews_page_response=requests.get(all_reviews_link,headers=HEADERS)
    soup_reviews=BeautifulSoup(reviews_page_response.text,"html.parser")

    reviews_ratings_info=soup_reviews.find('div',{'id':'filter-info-section'}).get_text().strip()
    #print("Reviews and Ratings info : "+reviews_ratings_info)

    reviews_listing=soup_reviews.find_all('div',{'data-hook':'review'})
    
    for list2 in reviews_listing:
        
        information={}
        
        information['asin']=asin
        information['product_title']=product_title
        information['product_url']=product_link
        information['product_store']=product_store
        information['product_rating']=product_rating
        information['product_image']=product_image
        
        customer_name=list2.find('div',{'class':'a-profile-content'}).get_text().strip()
        #print("Customer name is : "+customer_name)
        information['customer_name']=customer_name

        review_title=list2.find('a',{'class':'review-title-content'}).get_text().strip()
        #print("Review title : "+review_title)
        information['review_title']=review_title

        review_date=list2.find('span',{'class':'review-date'}).get_text().strip().replace("Reviewed in the United Kingdom on ","")
        #print("Review date : "+review_date)
        information['review_date']=review_date

        review_text=list2.find('span',{'class':'review-text-content'}).get_text().strip()
        #print("Review : "+review_text)
        information['review_text']=review_text

        l.append(information)
        

    try:
        next_page_reviews=soup_reviews.find('div',{'data-hook':'pagination-bar'}).find('li',{'class':'a-last'}).find('a')['href']
        next_page_reviews_link="https://www.amazon.co.uk"+next_page_reviews
        #print("Next Reviews Page : "+next_page_reviews_link)
        reviews_scrapper(next_page_reviews_link,product_title,product_image,asin,product_rating,product_store,product_link)
    except:
        print("***NO MORE REVIEWS***")





In [51]:

base_url1="https://www.amazon.co.uk/s?k=premier+housewares&page=7&qid=1712331382&ref=sr_pg_7"
#base_url1="https://www.amazon.co.uk/s?k=premier+housewares&qid=1711969341&ref=sr_pg_1"
scrapper123(base_url1)



https://www.amazon.co.uk/sspa/click?ie=UTF8&spc=MToyNzIxODczNDk0MzUxMDk1OjE3MTIzMzU1NDM6c3BfYXRmX25leHQ6MzAwMTU1NjU2NzE0MDMyOjowOjo&url=%2FHOSTECCO-Rectangle-Honeysuckle-Tablecloth-Washable%2Fdp%2FB0C8NNSD96%2Fref%3Dsr_1_289_sspa%3Fdib%3DeyJ2IjoiMSJ9.kVqvGlw993JX857wpZxVeBUOosA5OX8K2_ShEVm23GWASCY1k-KmEuglEf82JKYS5X8aIgjdeHLSlDMRL48AyePIsYyVgGYUehZGBtrFK-SkLYKLXRIkP_Usi1xdLrsDW_GYXf4ntbOMVMhU5kbabRoIM3ZfYLwEbxpjN4xoxXZsvYuspiwbF7kHqWtRrluSPR3JewMfhfwewyBpllPior37bvxqI5RPh4tE9-pEvmKPVMxcY1FndqycAl6H4SuzVE68JT4Exf4_-qcwZeps3w.V7d34i340TH_kaWNFwi9yISYEawN9v6ceBXAOhliw08%26dib_tag%3Dse%26keywords%3Dpremier%2Bhousewares%26qid%3D1712335543%26sr%3D8-289-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Product title is : HOSTECCO William Morris Rectangle Table Cover Honeysuckle Tablecloth Navy Blue Vintage Floral Tablecloth Washable for Dining Table Party Picnic 140x190 cm
https://www.amazon.co.uk/sspa/click?ie=UTF8&spc=MToyNzIxODczNDk0MzUxMDk1OjE3MTIzMzU1NDM6c3BfYXRmX25leHQ6MzAwMDU0N

In [93]:
df=pd.DataFrame(l,columns=['product_image','product_title','asin','product_rating','product_store','customer_name','review_title','review_date','review_text','product_url'])
df

,product_image,product_title,asin,product_rating,product_store,customer_name,review_title,review_date,review_text,product_url
0,https://m.media-amazon.com/images/I/71Kq818K1Z...,Premier Housewares 30 L Pedal Bin Slim Green K...,B073D4G1NC,3.6,Premier Store,Kamila M,5.0 out of 5 stars\nFive Stars,10 October 2015,Nice bin. Packed properly. Quick delivery. The...,https://www.amazon.co.uk/Premier-Housewares-St...
1,https://m.media-amazon.com/images/I/71Kq818K1Z...,Premier Housewares 30 L Pedal Bin Slim Green K...,B073D4G1NC,3.6,Premier Store,Chris,4.0 out of 5 stars\nPedal Bin,25 October 2012,The Bin arrived on time and was exactly aa des...,https://www.amazon.co.uk/Premier-Housewares-St...
2,https://m.media-amazon.com/images/I/71Kq818K1Z...,Premier Housewares 30 L Pedal Bin Slim Green K...,B073D4G1NC,3.6,Premier Store,Amazon Customer,3.0 out of 5 stars\nBin arrived with a dent in...,15 October 2015,Bin arrived with a dent in the lid but as it s...,https://www.amazon.co.uk/Premier-Housewares-St...
3,https://m.media-amazon.com/images/I/71Kq818K1Z...,Premier Housewares 30 L Pedal Bin Slim Green K...,B073D4G1NC,3.6,Premier Store,Mr W P Whitney-Smith,1.0 out of 5 stars\nVery poor quality - Lid Da...,12 October 2020,Product arrived in good time and packaged very...,https://www.amazon.co.uk/Premier-Housewares-St...
4,https://m.media-amazon.com/images/I/71Kq818K1Z...,Premier Housewares 30 L Pedal Bin Slim Green K...,B073D4G1NC,3.6,Premier Store,C G,2.0 out of 5 stars\nProduct arrived damaged. L...,26 April 2022,,https://www.amazon.co.uk/Premier-Housewares-St...
...,...,...,...,...,...,...,...,...,...,...
395,https://m.media-amazon.com/images/I/71F3o0CrL1...,Premier Housewares Casserole Glass Dishes - Se...,B004V2JF5Y,4.2,Premier Store,Jonathan,3.0 out of 5 stars\nThree Stars,15 May 2017,There is nothing wrong with them but they are ...,https://www.amazon.co.uk/Premier-Housewares-Ca...
396,https://m.media-amazon.com/images/I/71F3o0CrL1...,Premier Housewares Casserole Glass Dishes - Se...,B004V2JF5Y,4.2,Premier Store,MRSDOC6,2.0 out of 5 stars\nThis set are not nearly as...,8 March 2017,I bought these to replace a similar Pyrex set ...,https://www.amazon.co.uk/Premier-Housewares-Ca...
397,https://m.media-amazon.com/images/I/71F3o0CrL1...,Premier Housewares Casserole Glass Dishes - Se...,B004V2JF5Y,4.2,Premier Store,minimoo,5.0 out of 5 stars\nVery Useful,27 June 2020,Broke our last Pyrex dish like these during lo...,https://www.amazon.co.uk/Premier-Housewares-Ca...
398,https://m.media-amazon.com/images/I/71F3o0CrL1...,Premier Housewares Casserole Glass Dishes - Se...,B004V2JF5Y,4.2,Premier Store,Graham Kendrick,5.0 out of 5 stars\nExcellent,4 July 2020,Excellent produit just what I've been looking ...,https://www.amazon.co.uk/Premier-Housewares-Ca...


In [54]:
df.to_csv('amazon_product_reviews_page7.csv', index=False)